# Defensa del trabajo de fin de grado de informática:
### OPTIMIZACIÓN DE QUERIES DE BIG DATA CON SPARK EN AWS

La programación funcional es un paradigma de programación declarativa que se basa en las funciones matemáticas que nos permite tener códigos más limpios y estructurados, por tanto más modulares y todas las ventajas que llevan a ser modular. Como la disminución de la complejidad de los algoritmos, el ahorro en el tiempo de programación porque promueve la reusabilidad del código, y facilita la prueba y el mantenimiento.
 
Por el otro lado, con los avances tecnológicos en el campo de hardware, tenemos cada vez más núcleos para procesar, por tanto el software tiene que gestionar eficientemente los núcleos para sacar el máximo rendimiento. Pero con la programación imperativa, la que tenía el mayor mercado, surgen los problemas con la memoria compartida, lo que ralentizan los procesos. Pero con la programación funcional no hay ese problema por el uso de funciones puras que no producen efectos colaterales. Por eso la programación funcional está teniendo grandes crecimientos en el ámbito industrial en los últimos años. Por ejemplo la infraestructura de Whatsapp y la de Twitter.

Uno de los objetivos de este trabajo consiste en profundizar en la programación funcional, sobre todo en DSLs. 
También es un objetivo profundizar en el ecosistema de librerías de Scala: la librería Spark

En primer lugar importamos las librería que necesitamos

In [ ]:
import $file.sparksession
import sparksession._
import spark.implicits._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

In [ ]:
def readCsv(input: String ): DataFrame ={
    spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("data/" + input + ".csv")
}

La evaluación laziness de Dataset

In [ ]:
object Model{
    case class Disaster(Entity: String,
                   Year: Int,
                   Number: Int)
}

//dataset de prueba
val lista = List(Model.Disaster("All natural disasters", 2000, 50000),
        Model.Disaster("All natural disasters", 2001, 40000),
        Model.Disaster("Flood", 2000, 20000),
        Model.Disaster("Hurracan", 2000, 20000),
        Model.Disaster("fire", 2000, 10000),
        Model.Disaster("Flood", 2001, 20000),
        Model.Disaster("Starve", 2001, 20000))
val listDS = lista.toDS

In [ ]:
lista.filter(_.Number > 20000)
//los que tenga el elemento en la columna number
//mayor que 20000
val dsFilter = listDS.filter($"Number" > 20000)

In [ ]:
dsFilter.show

Pushed filter

In [ ]:
val ds : Dataset[String] = readCsv("GlobalLandTemperaturesByCountry").map(_.getAs[String]("Country"))

In [ ]:
val csv : DataFrame = readCsv("GlobalLandTemperaturesByCountry").filter($"Country" === "Denmark")

In [ ]:
val parquet: DataFrame = 
    spark.read.parquet("data/temperatureByCountry.parquet").filter($"Country" === "Denmark")

In [ ]:
ds.filter(_ == "Denmark").explain

In [ ]:
csv.explain

In [ ]:
parquet.explain

In [ ]:
csv.collect

In [ ]:
parquet.collect

PatitionFilter

In [ ]:
//filter sin partition
val sin: DataFrame = 
    spark.read.parquet("data/temperatureByCity.parquet")
            .filter($"Country" === "Denmark" && $"AverageTemperature" >10)

In [ ]:
//filter con partition
val con = spark.read.parquet("data/country.parquet")
            .filter($"Country" === "Denmark" && $"AverageTemperature" >10)

In [ ]:
sin.explain

In [ ]:
con.explain

In [ ]:
sin.collect

In [ ]:
con.collect

#### Gráfica con Plotly

Otro de los objetivo es reproducir unos resultados sobre unas bases de datos de desastres naturales, para ello hace falta hacer gráficas. En este caso he utilizado la librería plotly para conseguirlo

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.2`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

In [ ]:
val disasterNumber = readCsv("number-of-natural-disaster-events").drop($"Code")

In [ ]:
val aux = disasterNumber.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Number")

In [ ]:
val aux1 = aux.collect
Bar(aux1.map(_.getInt(0)).toSeq,
    aux1.map(_.getInt(1)).toSeq).plot()

Gráfica en Databricks

Enseñar los pasos en EMR